# Imputation and Data Consolidation

This notebook merges the processed match data with scrapped time data, imputes missing values, and consolidates the dataset.

In [1]:
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
import os

# Ensure the processed data directory exists
os.makedirs('../data/processed', exist_ok=True)

## Helper Functions

In [2]:
def trim_and_title(df):
    """Standardize column names and string values."""
    df.columns = df.columns.str.strip().str.title()
    for col in df.columns:
        if df[col].dtype == 'object':
            df[col] = df[col].str.strip().str.title()
    return df

def standardize_teams(df, cols):
    """Standardize IPL team names."""
    team_name_map = {
        'Deccan Chargers': 'Sunrisers Hyderabad',
        'Delhi Daredevils': 'Delhi Capitals',
        'Royal Challengers Bengaluru': 'Royal Challengers Bangalore',
        'Kings XI Punjab': 'Punjab Kings',
        'Kings Xi Punjab': 'Punjab Kings',
        'Rising Pune Supergiants': 'Rising Pune Supergiant',
        'Pune Warriors': 'Pune Warriors India'
    }
    for col in cols:
        if col in df.columns:
             df[col] = df[col].replace(team_name_map)
    return df

def clean_time(time_series):
    """Clean time strings efficiently."""
    if time_series.dtype != 'object':
        return time_series
        
    # Vectorized cleaning
    cleaned = time_series.astype(str).str.strip().str.lower()
    cleaned = cleaned.str.replace('.', '', regex=False)
    cleaned = cleaned.str.replace(' ', '', regex=False)
    cleaned = cleaned.str.replace('am', ' am', regex=False)
    cleaned = cleaned.str.replace('pm', ' pm', regex=False)
    
    # Remove leading zeros (e.g., '08:00' -> '8:00')
    return cleaned.apply(lambda x: x.lstrip('0') if pd.notna(x) else x)

def ist_to_local(df, time_col, city_col):
    """Convert IST time to local time based on City."""
    city_to_offset = {
        # South Africa (2009)
        'Cape Town': -3.5, 'Port Elizabeth': -3.5, 'Durban': -3.5,
        'Centurion': -3.5, 'East London': -3.5, 'Johannesburg': -3.5,
        'Kimberley': -3.5, 'Bloemfontein': -3.5,
        # UAE (2014, 2020, 2021)
        'Abu Dhabi': -1.5, 'Dubai': -1.5, 'Sharjah': -1.5
    }
    # Default offset is 0 for India
    
    def convert(row):
        time_str = row[time_col]
        city = row[city_col]
        
        if pd.isna(time_str):
            return None
            
        offset = city_to_offset.get(city, 0)
        if offset == 0:
            return time_str
            
        try:
            t = datetime.strptime(str(time_str), "%I:%M %p")
            local_t = t + timedelta(hours=offset)
            return local_t.strftime("%I:%M %p").lstrip("0").lower()
        except:
            return time_str

    return df.apply(convert, axis=1)

## Load Data

In [3]:
# Load processed parquet files
try:
    matches = pd.read_parquet('../data/processed/matches.parquet')
    print("Matches loaded from parquet.")
    # Standardize teams again to ensure consistency (e.g. handle 'Kings Xi Punjab')
    matches = standardize_teams(matches, ['Team1', 'Team2'])
except FileNotFoundError:
    print("Matches parquet not found. Please run P01_Pre_Processing.ipynb first.")

# Load scrapped data
try:
    all_matches = pd.read_csv('../data/scrapped/all_matches.csv')
    print("Scrapped matches loaded.")
except FileNotFoundError:
    print("Scrapped data not found.")

Matches loaded from parquet.
Scrapped matches loaded.


## Clean Scrapped Data

In [4]:
# Clean all_matches
all_matches = trim_and_title(all_matches)
all_matches = standardize_teams(all_matches, ['Team 1', 'Team 2'])
all_matches['Time'] = clean_time(all_matches['Time'])

## Fix Known Data Issues

In [5]:
# Issue 1: Missing Match 24 in 2012 in scrapped data
# Adding it manually to all_matches before merging
new_row = {
    'Match': 'Match 24',
    'Team 1': 'Chennai Super Kings',
    'Team 2': 'Pune Warriors India',
    'Date': pd.to_datetime('2012-04-19'),
    'Time': '8:00 pm',
    'Season': 2012
}

all_matches['Date'] = pd.to_datetime(all_matches['Date'], errors='coerce')

# Append new row
all_matches = pd.concat([all_matches, pd.DataFrame([new_row])], ignore_index=True)

# Issue 2: Incorrect date in matches dataset for Match 734043 (Reserve day used)
matches.loc[matches['Id'] == 734043, 'Date'] = pd.to_datetime('2014-05-28')

## Prepare for Merge

In [6]:
# Ensure Date Format is consistent (Date object)
matches['Date'] = pd.to_datetime(matches['Date']).dt.date
all_matches['Date'] = pd.to_datetime(all_matches['Date']).dt.date

# Create Match Key: Tuple of sorted team names to handle team order differences
matches['match_key'] = matches.apply(lambda x: tuple(sorted([x['Team1'], x['Team2']])), axis=1)
all_matches['match_key'] = all_matches.apply(lambda x: tuple(sorted([x['Team 1'], x['Team 2']])), axis=1)

# Rename columns in all_matches for clarity
all_matches = all_matches.rename(columns={'Match': 'Match_No'})

## Merge Data

In [7]:
# Merge matches with scrapped time and match number
merged = pd.merge(
    matches,
    all_matches[['Season', 'Date', 'match_key', 'Time', 'Match_No']],
    on=['Season', 'Date', 'match_key'],
    how='outer', # Using outer to catch the abandoned matches that might be in scrapped but not in matches
    indicator=True
)

# Split into matched and unmatched
matches_data = merged[merged['_merge'] == 'both'].copy()
matches_data.drop(columns=['_merge'], inplace=True)

# Rows in scrapped (all_matches) but not in matches (processed) -> Likely abandoned matches without results
abandoned_matches = merged[merged['_merge'] == 'right_only'].copy()
abandoned_matches.drop(columns=['_merge'], inplace=True)

print(f"Correctly Merged: {len(matches_data)}")
print(f"Abandoned/Unmatched (from scrapped): {len(abandoned_matches)}")

Correctly Merged: 1095
Abandoned/Unmatched (from scrapped): 11


## Incorporate Abandoned Matches

In [8]:
# Fill missing columns for abandoned matches
if not abandoned_matches.empty:
    # Extract Teams from match_key
    abandoned_matches['Team1'] = abandoned_matches['match_key'].apply(lambda x: x[0])
    abandoned_matches['Team2'] = abandoned_matches['match_key'].apply(lambda x: x[1])
    
    # Set default values for abandoned matches
    defaults = {
        'Match_Type': 'League', # Assumption, or could map if known
        'Toss_Winner': 'No Toss',
        'Toss_Decision': 'No Toss',
        'Winner': 'No Result',
        'Result': 'No Result',
        'Player_Of_Match': 'No Result',
        'Venue': 'Unknown', # Could imply from city if we had a map, or leave unknown
        'Result_Margin': -1,
        'Target_Runs': -1,
        'Target_Overs': -1,
        'Super_Over': 'N',
        'Method': 'No Result'
    }
    abandoned_matches = abandoned_matches.fillna(defaults)
    
    # Generate dummy IDs for these matches to avoid nulls (start from max id + 1)
    max_id = matches['Id'].max()
    abandoned_matches['Id'] = range(max_id + 1, max_id + 1 + len(abandoned_matches))

    # Concatenate back
    final_df = pd.concat([matches_data, abandoned_matches], ignore_index=True)
else:
    final_df = matches_data.copy()

## Impute Missing Values

In [9]:
# Impute Time (IST to Local)
final_df['Time'] = ist_to_local(final_df, 'Time', 'City')

# Validate imputation
print("Missing Times:", final_df['Time'].isna().sum())

Missing Times: 0


## Save Consolidated Data

In [10]:
# Select and Reorder columns
cols = [
    'Id', 'Season', 'Match_No', 'Date', 'Time', 'City', 'Venue', 
    'Team1', 'Team2', 'Toss_Winner', 'Toss_Decision', 
    'Winner', 'Result', 'Result_Margin', 'Target_Runs', 'Target_Overs', 
    'Player_Of_Match', 'Match_Type', 'Super_Over', 'Method', 
    'Umpire1', 'Umpire2'
]

# Ensure all columns exist
for col in cols:
    if col not in final_df.columns:
        final_df[col] = None

final_df = final_df[cols].sort_values(['Season', 'Date']).reset_index(drop=True)

# Save
try:
    final_df.to_parquet('../data/processed/matches_imputed.parquet', index=False)
    print("Success: Consolidated data saved to ../data/processed/matches_imputed.parquet")
except Exception as e:
    print(f"Error saving parquet: {e}")

Success: Consolidated data saved to ../data/processed/matches_imputed.parquet


In [11]:
final_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1106 entries, 0 to 1105
Data columns (total 22 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   Id               1106 non-null   float64
 1   Season           1106 non-null   int64  
 2   Match_No         1106 non-null   object 
 3   Date             1106 non-null   object 
 4   Time             1106 non-null   object 
 5   City             1095 non-null   object 
 6   Venue            1106 non-null   object 
 7   Team1            1106 non-null   object 
 8   Team2            1106 non-null   object 
 9   Toss_Winner      1106 non-null   object 
 10  Toss_Decision    1106 non-null   object 
 11  Winner           1106 non-null   object 
 12  Result           1106 non-null   object 
 13  Result_Margin    1106 non-null   float64
 14  Target_Runs      1106 non-null   float64
 15  Target_Overs     1106 non-null   float64
 16  Player_Of_Match  1106 non-null   object 
 17  Match_Type    